# Development of GravNet Architecture

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
from pprint import pprint as pp
from time import time as tt
import inspect
import logging

# External imports
import matplotlib.pyplot as plt
import matplotlib.colors
from sklearn.decomposition import PCA
from sklearn.metrics import auc, f1_score
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from mpl_toolkits.mplot3d import Axes3D
from itertools import permutations
from itertools import chain

from torch.nn import Linear
from torch_scatter import scatter, segment_csr, scatter_add
from torch_geometric.nn.conv import MessagePassing
from torch_cluster import knn_graph, radius_graph
import trackml.dataset
import torch_geometric

import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import WandbLogger
from torch.utils.checkpoint import checkpoint

# Limit CPU usage on Jupyter
os.environ['OMP_NUM_THREADS'] = '4'

# Pick up local packages
sys.path.append('..')

# Local imports
from prepare import select_hits
from toy_utils import *
from lightning_modules.GravNet.utils import evaluate_set_metrics, get_metrics, model_evaluation
from pytorch_lightning.loggers import WandbLogger
%matplotlib inline

# Get rid of RuntimeWarnings, gross
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
logging.basicConfig(level=logging.INFO)

In [63]:
logging.basicConfig(level=logging.WARNING)

# Load Data

TODO:
* Implement GravNet lightning module with data loading
* Enforce pT cut > 1GeV
* Ensure this is CodaLab
* Just use preprocessed data!

# Embedding Benchmark

TODO:
* Quickly implement embedding module
* Train
* Run test inference and see best f1

In [10]:
from lightning_modules.Embedding.Models.layerless_embedding import LayerlessEmbedding

In [11]:
with open("../lightning_modules/Embedding/train_coda_small_embedding.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [12]:
model = LayerlessEmbedding(hparams)

In [13]:
trainer = Trainer(gpus=1, max_epochs=6)

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


### LR Analysis

In [ ]:
lr_finder = trainer.tuner.lr_find(model)

In [ ]:
# Results can be found in
lr_finder.results

In [ ]:
fig = lr_finder.plot(suggest=True)
fig.show()

In [ ]:
new_lr = lr_finder.suggestion()

# update hparams of the model
model.hparams.lr = new_lr

### Training

In [14]:
trainer.fit(model)

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021000
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021001
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021002
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021003
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021004
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021006
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021007
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021008
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021080
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021081
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021082
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021083
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021084
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021085
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021086
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021087
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021088
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Efficiency: 0.8784927129745483
INFO:root:Purity: 0.010595016181468964
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8834808468818665
INFO:root:Purity: 0.01054308284074068
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']


INFO:root:Efficiency: 0.8868021965026855
INFO:root:Purity: 0.010695232078433037
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8908554315567017
INFO:root:Purity: 0.010631089098751545
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.8946163654327393
INFO:root:Purity: 0.010744397528469563
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.9111727476119995
INFO:root:Purity: 0.010970826260745525
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.9047814607620239
INFO:root:Purity: 0.011155054904520512
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.9242463111877441
INFO:root:Purity: 0.011323874816298485
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/track

INFO:root:Efficiency: 0.6634808778762817
INFO:root:Purity: 0.074409618973732
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.6626632809638977
INFO:root:Purity: 0.08366067707538605
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.6564975380897522
INFO:root:Purity: 0.07160695642232895
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.6590666174888611
INFO:root:Purity: 0.07535241544246674
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.683357298374176
INFO:root:Purity: 0.07898669689893723
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.6795991659164429
INFO:root:Purity: 0.10175294429063797
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codala

INFO:root:Efficiency: 0.7427406311035156
INFO:root:Purity: 0.13552449643611908
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.7491922974586487
INFO:root:Purity: 0.15540727972984314
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.7451732754707336
INFO:root:Purity: 0.12930728495121002
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.756820023059845
INFO:root:Purity: 0.13771413266658783
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.7647581696510315
INFO:root:Purity: 0.1526738405227661
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.7550531029701233
INFO:root:Purity: 0.18404242396354675
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codal

INFO:root:Efficiency: 0.7947282195091248
INFO:root:Purity: 0.16938267648220062
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.7977244257926941
INFO:root:Purity: 0.19614209234714508
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.7891708016395569
INFO:root:Purity: 0.15842433273792267
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.8067665100097656
INFO:root:Purity: 0.16854308545589447
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8053218126296997
INFO:root:Purity: 0.19641636312007904
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.8050702214241028
INFO:root:Purity: 0.23120249807834625
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Efficiency: 0.8585864901542664
INFO:root:Purity: 0.22532151639461517
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8569321632385254
INFO:root:Purity: 0.260972797870636
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.8599628806114197
INFO:root:Purity: 0.21813929080963135
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.8655388951301575
INFO:root:Purity: 0.2185843139886856
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8616184592247009
INFO:root:Purity: 0.2633328437805176
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.8645083904266357
INFO:root:Purity: 0.2984183430671692
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/

INFO:root:Efficiency: 0.8794058561325073
INFO:root:Purity: 0.2696658670902252
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8840426802635193
INFO:root:Purity: 0.3129693269729614
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.8822401165962219
INFO:root:Purity: 0.2646900415420532
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.8895426392555237
INFO:root:Purity: 0.2613238990306854
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8811136484146118
INFO:root:Purity: 0.31323865056037903
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.8875470757484436
INFO:root:Purity: 0.34607934951782227
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab

1

In [15]:
trainer.test()

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


INFO:root:Efficiency: 0.8836482167243958
INFO:root:Purity: 0.30355098843574524
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021132']
INFO:root:Efficiency: 0.8918190598487854
INFO:root:Purity: 0.3033408224582672
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021129']
INFO:root:Efficiency: 0.8811609148979187
INFO:root:Purity: 0.27055278420448303
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021056']
INFO:root:Efficiency: 0.8865882754325867
INFO:root:Purity: 0.257443368434906
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021118']
INFO:root:Efficiency: 0.8956884741783142
INFO:root:Purity: 0.2842729091644287
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021119']
INFO:root:Efficiency: 0.8714001774787903
INFO:root:Purity: 0.27958184480667114
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.4110, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,   251,   273, ..., 11792, 11807, 11989]]),
 'truth': array([False, False,  True, ..., False,  True, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.4199, device='cuda:0'),
 'preds': array([[    0,     0,     1, ..., 10328, 10328, 10328],
       [   12,  1966,     2, ..., 10222, 10223, 10235]]),
 'truth': array([False,  True, False, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

[{'loss': 0.4109977185726166,
  'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
         [    1,   251,   273, ..., 11792, 11807, 11989]]),
  'truth': array([False, False,  True, ..., False,  True, False]),
  'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
         [ 606,  662,  606, ..., 3861, 4898, 6374]])},
 {'loss': 0.4198637902736664,
  'preds': array([[    0,     0,     1, ..., 10328, 10328, 10328],
         [   12,  1966,     2, ..., 10222, 10223, 10235]]),
  'truth': array([False,  True, False, ..., False, False, False]),
  'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
         [4038, 4068, 4771, ..., 8278, 9154, 9520]])},
 {'loss': 0.420266717672348,
  'preds': array([[    0,     0,     0, ..., 14294, 14294, 14294],
         [ 1221,  1264,  3066, ..., 14235, 14287, 14289]]),
  'truth': array([ True,  True, False, ..., False, False, False]),
  'truth_graph': array([[ 3121,  3121,  3310, ..., 11432, 12165, 12165],
         [ 6

### Testing

In [15]:
model_evaluation(model, trainer, "eff", 0.99)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


INFO:root:Efficiency: 0.33355987071990967
INFO:root:Purity: 0.44013988971710205
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021132']
INFO:root:Efficiency: 0.31915414333343506
INFO:root:Purity: 0.42346441745758057
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021129']
INFO:root:Efficiency: 0.3194931149482727
INFO:root:Purity: 0.396162211894989
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021056']
INFO:root:Efficiency: 0.3256584405899048
INFO:root:Purity: 0.38754189014434814
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021118']
INFO:root:Efficiency: 0.3308165967464447
INFO:root:Purity: 0.4052409529685974
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021119']
INFO:root:Efficiency: 0.3199281692504883
INFO:root:Purity: 0.38690659403800964
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-coda

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.6059, device='cuda:0'),
 'preds': array([[    0,     1,     3, ..., 12136, 12136, 12138],
       [    1,     1,     4, ..., 11802, 12018, 11807]]),
 'truth': array([False, False, False, ...,  True, False,  True]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.6160, device='cuda:0'),
 'preds': array([[    0,     1,     1, ..., 10326, 10327, 10328],
       [   12,     2,     6, ..., 10044, 10046, 10048]]),
 'truth': array([False, False, False, ...,  True,  True,  True]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.4068162252314028 0.3304591186442749


...

0.038565392966745125 0.9899941780485045
Seed solver complete, radius: 2.4103622551843893


INFO:root:Efficiency: 0.9842327237129211
INFO:root:Purity: 0.042529698461294174
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021132']
INFO:root:Efficiency: 0.9937949180603027
INFO:root:Purity: 0.04311801493167877
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021129']
INFO:root:Efficiency: 0.9873277544975281
INFO:root:Purity: 0.036171671003103256
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021056']
INFO:root:Efficiency: 0.9922698736190796
INFO:root:Purity: 0.032343991100788116
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021118']
INFO:root:Efficiency: 0.9907940626144409
INFO:root:Purity: 0.038385845720767975
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021119']
INFO:root:Efficiency: 0.9856968522071838
INFO:root:Purity: 0.039754971861839294
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackm

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.0606, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,     2,    18, ..., 12000, 12126, 12128]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.0632, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
       [   12,   754,  1236, ..., 10235, 10242, 10261]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

0.03854994221677041 0.989997116114082


((0.989997116114082, 0.03854994221677041, 0.07419149692498578),
 2.4103622551843893)

In [74]:
mean_f1s, mean_efficiencies, mean_purities = [], [], []

for r_test in np.arange(1.0, 1.1, 0.1):
    
    mean_efficiency, mean_purity, mean_f1 = evaluate_set_metrics(r_test, model, trainer)
    mean_efficiencies.append(mean_efficiency), mean_purities.append(mean_purity), mean_f1s.append(mean_f1)
    
    print("Mean efficiency: {:.2f}%".format(mean_efficiency*100))
    print("Mean purity: {:.2f}%".format(mean_purity*100))
    print("Mean F1: {:.2f}%".format(mean_f1*100))

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.4169, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,   273,   284, ..., 11245, 11628, 11807]]),
 'truth': array([False,  True, False, ..., False, False,  True]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.4221, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
       [   12,  1966,  3669, ...,  9973, 10048, 10209]]),
 'truth': array([False,  True, False, ..., False,  True, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.3422392651557339 0.9139188275046294
Mean efficiency: 91.39%
Mean purity: 34.22%
Mean F1: 50.95%


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.3569, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,   273,   284, ..., 11628, 11807, 11989]]),
 'truth': array([False,  True, False, ..., False,  True, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.3600, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
       [   12,  1966,  3669, ..., 10222, 10223, 10235]]),
 'truth': array([False,  True, False, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

0.29214248736765147 0.9385322925332436
Mean efficiency: 93.85%
Mean purity: 29.21%
Mean F1: 45.17%


# GravNet Module

TODO:
* Write up GravNet module that should be an embedding killer
* Understand if co-training metric and edge classification is relevant

In [17]:
from lightning_modules.GravNet.Models.vanilla_gravnet import VanillaGravNet

with open("../lightning_modules/GravNet/train_coda_small_gravnet.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)
        
model = VanillaGravNet(hparams)
wandb_logger = WandbLogger(project='GravNetLite')
wandb_logger.watch(model)
trainer = Trainer(gpus=1, max_epochs=10, logger=wandb_logger)

### KNN Debug

Todo:

- [ ] Copy in KNN maker code
- [ ] Apply model to random data on GPU
- [ ] Try different sizes of KNN

In [12]:
import faiss
import faiss.contrib.torch_utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [13]:
def build_knn(spatial, k):
    
    if device == "cuda":
        res = faiss.StandardGpuResources()
        _, I = faiss.knn_gpu(res, spatial, spatial, k)
    elif device == "cpu":
        index = faiss.IndexFlatL2(spatial.shape[1])
        index.add(spatial)
        _, I = index.search(spatial, k)
        
    I = I[:, 1:]
    
    ind = torch.Tensor.repeat_interleave(torch.arange(I.shape[0], device=device), I.shape[1])
    I = torch.flatten(I)
    
    edge_list = torch.stack([ind, I])
    
    return edge_list

In [83]:
torch.cuda.reset_max_memory_allocated()

In [84]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  240155 KB |  240155 KB |     854 GB |     854 GB |
|       from large pool |  231378 KB |  231378 KB |     772 GB |     772 GB |
|       from small pool |    8776 KB |    8776 KB |      81 GB |      81 GB |
|---------------------------------------------------------------------------|
| Active memory         |  240155 KB |  240155 KB |     854 GB |     854 GB |
|       from large pool |  231378 KB |  231378 KB |     772 GB |

In [85]:
model = VanillaGravNet(hparams).to(device)
model.train()

VanillaGravNet(
  (space_encoder): Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=32, bias=True)
  )
  (feature_encoder): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (5): ReLU()
  )
  (space_network): Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (5): ReLU()
  )
  (feature_network): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): LayerNorm((64,), eps=1e-05, e

In [86]:
model.setup("fit")

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021000
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021001
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021002
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021003
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021004
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021006
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021007
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021008
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

In [87]:
batch = model.trainset[0].to(device)

In [89]:
spatial = model(batch.x)

torch.Size([12083, 32])


In [91]:
spatial

tensor([[-0.0448, -0.0567, -0.3559,  ...,  0.5205,  0.0851, -0.7749],
        [-0.2232, -0.1707,  0.0106,  ...,  0.3912,  0.3089, -0.3286],
        [-0.1268, -0.3000,  0.2403,  ...,  0.2021,  0.0976, -0.1908],
        ...,
        [ 0.0066,  0.0333, -0.6734,  ...,  0.3288,  0.1993, -0.7898],
        [ 0.0097,  0.0018, -0.6540,  ...,  0.3344,  0.1209, -0.7924],
        [ 0.0078, -0.0013, -0.6530,  ...,  0.3381,  0.1176, -0.7967]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [98]:
%%time
for i in range(100):
    e_spatial = build_knn(spatial, 500)

CPU times: user 2.04 s, sys: 14.6 s, total: 16.6 s
Wall time: 16.6 s


In [109]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  469763 KB |     960 MB |     918 GB |     917 GB |
|       from large pool |  458289 KB |     944 MB |     835 GB |     834 GB |
|       from small pool |   11474 KB |      23 MB |      82 GB |      82 GB |
|---------------------------------------------------------------------------|
| Active memory         |  469763 KB |     960 MB |     918 GB |     917 GB |
|       from large pool |  458289 KB |     944 MB |     835 GB |

In [102]:
res = faiss.StandardGpuResources()
_, I = faiss.knn_gpu(res, spatial, spatial, 500)

### LR Analysis

In [ ]:
lr_finder = trainer.tuner.lr_find(model)

In [ ]:
# Results can be found in
lr_finder.results

In [ ]:
fig = lr_finder.plot(suggest=True)
fig.show()

In [ ]:
new_lr = lr_finder.suggestion()

# update hparams of the model
model.hparams.lr = new_lr

### Training

In [20]:
%%time
trainer.fit(model)

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021000
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021001
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021002
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021003
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021004
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021006
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021007
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021008
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021080
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021081
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021082
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021083
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021084
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021085
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021086
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021087
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021088
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

../lightning_modules/GravNet/gravnet_base.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  eff = torch.tensor(cluster_true_positive / cluster_true)
../lightning_modules/GravNet/gravnet_base.py:177: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pur = torch.tensor(cluster_true_positive / cluster_positive)
INFO:root:Efficiency: 0.7765873074531555
INFO:root:Purity: 0.009365991689264774
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8097345232963562
INFO:root:Purity: 0.009663025848567486
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']


INFO:root:Efficiency: 0.3077859580516815
INFO:root:Purity: 0.045024264603853226
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.30306223034858704
INFO:root:Purity: 0.048783522099256516
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.2847772240638733
INFO:root:Purity: 0.043202344328165054
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.3147900402545929
INFO:root:Purity: 0.04907643049955368
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.31698474287986755
INFO:root:Purity: 0.05134967342019081
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.31252139806747437
INFO:root:Purity: 0.053986478596925735
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trac

INFO:root:Efficiency: 0.5538442730903625
INFO:root:Purity: 0.12499313056468964
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.5634218454360962
INFO:root:Purity: 0.13803426921367645
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.5560643672943115
INFO:root:Purity: 0.12134061008691788
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.5643219947814941
INFO:root:Purity: 0.12879793345928192
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.5881386995315552
INFO:root:Purity: 0.14343148469924927
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.5715141892433167
INFO:root:Purity: 0.15820668637752533
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

...

INFO:root:Efficiency: 0.8002069592475891
INFO:root:Purity: 0.22498930990695953
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8056609034538269
INFO:root:Purity: 0.25618061423301697
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.7952351570129395
INFO:root:Purity: 0.21937520802021027
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.8009494543075562
INFO:root:Purity: 0.22087213397026062
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8052534461021423
INFO:root:Purity: 0.2613154649734497
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.7923946380615234
INFO:root:Purity: 0.28381237387657166
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-coda

INFO:root:Efficiency: 0.8509161472320557
INFO:root:Purity: 0.2764635980129242
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.859390377998352
INFO:root:Purity: 0.31020408868789673
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.8538985252380371
INFO:root:Purity: 0.2674691379070282
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.85965496301651
INFO:root:Purity: 0.2706224024295807
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8643546104431152
INFO:root:Purity: 0.31256338953971863
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.865878701210022
INFO:root:Purity: 0.3368877172470093
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/tra

INFO:root:Efficiency: 0.8672916293144226
INFO:root:Purity: 0.2927987277507782
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8722432851791382
INFO:root:Purity: 0.3259580135345459
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.8629331588745117
INFO:root:Purity: 0.2831932008266449
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.8791789412498474
INFO:root:Purity: 0.27913641929626465
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8729051351547241
INFO:root:Purity: 0.3302878141403198
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.8875470757484436
INFO:root:Purity: 0.3490988612174988
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/


CPU times: user 1min 20s, sys: 3min 26s, total: 4min 47s
Wall time: 4min 36s


1

In [12]:
%%time
trainer.fit(model)

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021000
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021001
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021002
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021003
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021004
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021006
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021007
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021008
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021080
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021081
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021082
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021083
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021084
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021085
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021086
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021087
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021088
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Efficiency: 0.5020393133163452
INFO:root:Purity: 0.12214701622724533
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.5191038250923157
INFO:root:Purity: 0.1362672597169876
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']


INFO:root:Efficiency: 0.7163206934928894
INFO:root:Purity: 0.20351441204547882
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.7255232334136963
INFO:root:Purity: 0.23307761549949646
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.6879331469535828
INFO:root:Purity: 0.19777268171310425
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.7239903807640076
INFO:root:Purity: 0.20328164100646973
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.7365756630897522
INFO:root:Purity: 0.23517078161239624
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021048']
INFO:root:Efficiency: 0.723878026008606
INFO:root:Purity: 0.24880044162273407
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-coda

CPU times: user 39 s, sys: 52.2 s, total: 1min 31s
Wall time: 1min 28s


1

In [21]:
trainer.test()

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


INFO:root:Efficiency: 0.8731819987297058
INFO:root:Purity: 0.32209381461143494
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021132']
INFO:root:Efficiency: 0.8901861310005188
INFO:root:Purity: 0.3268089294433594
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021129']
INFO:root:Efficiency: 0.8688974380493164
INFO:root:Purity: 0.29649484157562256
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021056']
INFO:root:Efficiency: 0.8728949427604675
INFO:root:Purity: 0.2840178310871124
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021118']
INFO:root:Efficiency: 0.8797271251678467
INFO:root:Purity: 0.30483773350715637
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021119']
INFO:root:Efficiency: 0.8599191904067993
INFO:root:Purity: 0.30124029517173767
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codal

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'current_lr': tensor(0.0010),
 'eff': tensor(0.8759),
 'pur': tensor(0.3012),
 'train_loss': tensor(0.3238, device='cuda:0'),
 'val_loss': tensor(0.4425, device='cuda:0')}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.4397, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,   273,   284, ..., 11628, 11807, 11989]]),
 'truth': array([False,  True, False, ..., False,  True, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:2 TEST RESULTS
{'loss': tensor(0.4391, device='cuda:0'),
 'preds': array([[    0,     0,     1, ..., 10328, 10328, 10328],
       [   12,  1966,     2, ..., 10222, 10223, 

[{'train_loss': 0.3237735629081726,
  'val_loss': 0.44253265857696533,
  'eff': 0.875904381275177,
  'pur': 0.3011761009693146,
  'current_lr': 0.0010000000474974513},
 {'loss': 0.4397120177745819,
  'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
         [    1,   273,   284, ..., 11628, 11807, 11989]]),
  'truth': array([False,  True, False, ..., False,  True, False]),
  'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
         [ 606,  662,  606, ..., 3861, 4898, 6374]])},
 {'loss': 0.43914151191711426,
  'preds': array([[    0,     0,     1, ..., 10328, 10328, 10328],
         [   12,  1966,     2, ..., 10222, 10223, 10235]]),
  'truth': array([False,  True, False, ..., False, False, False]),
  'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
         [4038, 4068, 4771, ..., 8278, 9154, 9520]])},
 {'loss': 0.4373224377632141,
  'preds': array([[    0,     0,     0, ..., 14294, 14294, 14294],
         [ 1221,  1264,  2016, ..., 14235,

In [9]:
test = [100, 20, 20]

In [13]:
b = np.split(np.arange(0, 140), [sum(test[:x+1]) for x in range(0,len(test))])

In [17]:
[i.tolist() for i in b]

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99],
 [100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119],
 [120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139],
 []]

In [15]:
torch.cuda.reset_max_memory_allocated()

/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/torch/cuda/memory.py:234: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)


In [10]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    7288 KB |    1421 MB |    3450 GB |    3450 GB |
|       from large pool |       0 KB |    1395 MB |    3389 GB |    3389 GB |
|       from small pool |    7288 KB |      35 MB |      60 GB |      60 GB |
|---------------------------------------------------------------------------|
| Active memory         |    7288 KB |    1421 MB |    3450 GB |    3450 GB |
|       from large pool |       0 KB |    1395 MB |    3389 GB |

### Testing

In [10]:
model.eval()
model.validation_step(model.valset[0].to(device), 0)

INFO:root:Efficiency: 0.21574237942695618
INFO:root:Purity: 0.04584141820669174
INFO:root:/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040


tensor(0.4003, device='cuda:0', grad_fn=<MeanBackward0>)

In [8]:
model.hparams.r_test

1

In [9]:
trainer.test()

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


INFO:root:Efficiency: 0.21639254689216614
INFO:root:Purity: 0.04851144179701805
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021132']
INFO:root:Efficiency: 0.21783147752285004
INFO:root:Purity: 0.05128106474876404
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021129']
INFO:root:Efficiency: 0.2059682309627533
INFO:root:Purity: 0.04139622673392296
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021056']
INFO:root:Efficiency: 0.21075589954853058
INFO:root:Purity: 0.040681254118680954
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021118']
INFO:root:Efficiency: 0.22617392241954803
INFO:root:Purity: 0.04882826283574104
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021119']
INFO:root:Efficiency: 0.21313577890396118
INFO:root:Purity: 0.04445068538188934
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/track

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.4132, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,    50,   107, ..., 12071, 12080, 12120]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.4083, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
       [   12,  3592,  3642, ..., 10263, 10279, 10296]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

[{'loss': 0.41321906447410583,
  'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
         [    1,    50,   107, ..., 12071, 12080, 12120]]),
  'truth': array([False, False, False, ..., False, False, False]),
  'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
         [ 606,  662,  606, ..., 3861, 4898, 6374]])},
 {'loss': 0.40834227204322815,
  'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
         [   12,  3592,  3642, ..., 10263, 10279, 10296]]),
  'truth': array([False, False, False, ..., False, False, False]),
  'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
         [4038, 4068, 4771, ..., 8278, 9154, 9520]])},
 {'loss': 0.4102642238140106,
  'preds': array([[    0,     0,     0, ..., 14294, 14294, 14294],
         [  917,  1552,  1650, ..., 14235, 14287, 14289]]),
  'truth': array([False, False, False, ..., False, False, False]),
  'truth_graph': array([[ 3121,  3121,  3310, ..., 11432, 12165, 12165],
         

In [9]:
model_evaluation(model, trainer, "eff", 0.99)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.4793, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,    18,    50, ..., 12004, 12022, 12128]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.4859, device='cuda:0'),
 'preds': array([[    1,     1,     1, ..., 10328, 10328, 10328],
       [    2,     6,     7, ..., 10298, 10302, 10323]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.04521267804865519 0.11813487924851136


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.1883, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,    18,    50, ..., 12120, 12127, 12128]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.1955, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
       [   12,  3692,  3973, ..., 10313, 10317, 10323]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

...

0.0113021839428851 0.9362145738160079


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.0365, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,     2,    18, ..., 12127, 12128, 12137]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.0348, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
       [    8,    12,    45, ..., 10313, 10317, 10323]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.011302232045165382 0.9362182146363566


RuntimeError: value cannot be converted to type float without overflow: 3.85856e+43

# GravNetLite Module

TODO:
* Write up GravNet module that should be an embedding killer
* Understand if co-training metric and edge classification is relevant

In [8]:
from lightning_modules.GravNet.Models.gravnet_lite import GravNetLite
from lightning_modules.GravNet.Models.gravnet_lite import CheckpointGravNetLite

with open("../lightning_modules/GravNet/train_coda_small_gravnet.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)
        
# model = GravNetLite(hparams)
model = CheckpointGravNetLite(hparams)

wandb_logger = WandbLogger(project='GravNetLite')
wandb_logger.watch(model)
trainer = Trainer(gpus=1, max_epochs=10, logger=wandb_logger)

wandb: wandb version 0.10.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


### Training

In [9]:
%%time
trainer.fit(model)

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021000
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021001
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021002
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021003
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021004
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021006
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021007
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021008
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021080
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021081
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021082
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021083
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021084
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021085
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021086
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021087
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021088
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Efficiency: 0.7456930875778198
INFO:root:Purity: 0.008993963710963726
INFO:root:Loss: 0.9671688675880432
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.7558645606040955
INFO:root:Purity: 0.009020279161632061
INFO:root:Loss: 0.9651855230331421
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']


INFO:root:Efficiency: 0.17227734625339508
INFO:root:Purity: 0.035877738147974014
INFO:root:Loss: 0.44004666805267334
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.18991431593894958
INFO:root:Purity: 0.04306966811418533
INFO:root:Loss: 0.43876850605010986
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.188490092754364
INFO:root:Purity: 0.038034588098526
INFO:root:Loss: 0.4411172568798065
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.1819336712360382
INFO:root:Purity: 0.03888142481446266
INFO:root:Loss: 0.45055022835731506
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.19590942561626434
INFO:root:Purity: 0.04326676204800606
INFO:root:Loss: 0.4414866864681244
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trac

INFO:root:Efficiency: 0.3424849212169647
INFO:root:Purity: 0.09341945499181747
INFO:root:Loss: 0.40597254037857056
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.3531394898891449
INFO:root:Purity: 0.10405413806438446
INFO:root:Loss: 0.40657222270965576
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.345730185508728
INFO:root:Purity: 0.09213846176862717
INFO:root:Loss: 0.41328689455986023
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.3290987014770508
INFO:root:Purity: 0.09654956310987473
INFO:root:Loss: 0.4187864065170288
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.3502291440963745
INFO:root:Purity: 0.10859651863574982
INFO:root:Loss: 0.4083796739578247
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackm

...

INFO:root:Efficiency: 0.8698484301567078
INFO:root:Purity: 0.33270466327667236
INFO:root:Loss: 0.4352751076221466
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8799691200256348
INFO:root:Purity: 0.3685433566570282
INFO:root:Loss: 0.4354456961154938
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.8748143315315247
INFO:root:Purity: 0.31574833393096924
INFO:root:Loss: 0.4361293613910675
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.879981279373169
INFO:root:Purity: 0.3187994956970215
INFO:root:Loss: 0.4350833296775818
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8769409656524658
INFO:root:Purity: 0.3749634325504303
INFO:root:Loss: 0.43063074350357056
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Efficiency: 0.8755707144737244
INFO:root:Purity: 0.3345039188861847
INFO:root:Loss: 0.4301336705684662
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8828487396240234
INFO:root:Purity: 0.36678239703178406
INFO:root:Loss: 0.4308670163154602
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.876980185508728
INFO:root:Purity: 0.31988805532455444
INFO:root:Loss: 0.43366479873657227
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.888405978679657
INFO:root:Purity: 0.3226487934589386
INFO:root:Loss: 0.4334125816822052
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8862439393997192
INFO:root:Purity: 0.3747866451740265
INFO:root:Loss: 0.428082674741745
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codal

INFO:root:Efficiency: 0.8749010562896729
INFO:root:Purity: 0.34041544795036316
INFO:root:Loss: 0.43334993720054626
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021040']
INFO:root:Efficiency: 0.8761764168739319
INFO:root:Purity: 0.3747146427631378
INFO:root:Loss: 0.4348010718822479
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005']
INFO:root:Efficiency: 0.8743811845779419
INFO:root:Purity: 0.3195386826992035
INFO:root:Loss: 0.43433135747909546
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021133']
INFO:root:Efficiency: 0.8808504939079285
INFO:root:Purity: 0.3238605558872223
INFO:root:Loss: 0.437208890914917
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021025']
INFO:root:Efficiency: 0.8811136484146118
INFO:root:Purity: 0.3788640797138214
INFO:root:Loss: 0.433553010225296
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-coda


CPU times: user 3min 12s, sys: 21min 33s, total: 24min 45s
Wall time: 25min 4s


1

In [10]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  193630 KB |    2524 MB |   19165 GB |   19165 GB |
|       from large pool |  191369 KB |    2518 MB |   19081 GB |   19080 GB |
|       from small pool |    2261 KB |      16 MB |      84 GB |      84 GB |
|---------------------------------------------------------------------------|
| Active memory         |  193630 KB |    2524 MB |   19165 GB |   19165 GB |
|       from large pool |  191369 KB |    2518 MB |   19081 GB |

In [15]:
torch.cuda.reset_max_memory_allocated()

/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/torch/cuda/memory.py:234: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)


### Testing

In [18]:
trainer.test(ckpt_path=None)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


INFO:root:Efficiency: 0.9774364829063416
INFO:root:Purity: 0.09650924056768417
INFO:root:Loss: 0.1662280559539795
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021132']
INFO:root:Efficiency: 0.9848138689994812
INFO:root:Purity: 0.10110729932785034
INFO:root:Loss: 0.17225468158721924
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021129']
INFO:root:Efficiency: 0.9786264896392822
INFO:root:Purity: 0.08465004712343216
INFO:root:Loss: 0.1614999771118164
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021056']
INFO:root:Efficiency: 0.9801778197288513
INFO:root:Purity: 0.07861998677253723
INFO:root:Loss: 0.15623630583286285
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021118']
INFO:root:Efficiency: 0.979468822479248
INFO:root:Purity: 0.09146910905838013
INFO:root:Loss: 0.165895476937294
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.1662, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    0,   251,   273, ..., 11792, 11807, 11989]]),
 'truth': array([False, False,  True, ..., False,  True, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.1723, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
       [   12,  1496,  1966, ..., 10209, 10222, 10261]]),
 'truth': array([False, False,  True, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

[{'loss': 0.1662280559539795,
  'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
         [    0,   251,   273, ..., 11792, 11807, 11989]]),
  'truth': array([False, False,  True, ..., False,  True, False]),
  'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
         [ 606,  662,  606, ..., 3861, 4898, 6374]])},
 {'loss': 0.17225468158721924,
  'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
         [   12,  1496,  1966, ..., 10209, 10222, 10261]]),
  'truth': array([False, False,  True, ..., False, False, False]),
  'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
         [4038, 4068, 4771, ..., 8278, 9154, 9520]])},
 {'loss': 0.1614999771118164,
  'preds': array([[    0,     0,     0, ..., 14294, 14294, 14294],
         [ 1221,  1264,  2016, ..., 14263, 14287, 14289]]),
  'truth': array([ True,  True, False, ..., False, False, False]),
  'truth_graph': array([[ 3121,  3121,  3310, ..., 11432, 12165, 12165],
         [

In [11]:
metrics, radius = model_evaluation(model, trainer, "eff", 0.99)
eff, pur, f1 = metrics

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


INFO:root:Efficiency: 0.8250645399093628
INFO:root:Purity: 0.4307255744934082
INFO:root:Loss: 0.48814156651496887
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021132']
INFO:root:Efficiency: 0.8251959681510925
INFO:root:Purity: 0.4351588785648346
INFO:root:Loss: 0.49622097611427307
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021129']
INFO:root:Efficiency: 0.820193886756897
INFO:root:Purity: 0.39846232533454895
INFO:root:Loss: 0.4909903109073639
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021056']
INFO:root:Efficiency: 0.8309866786003113
INFO:root:Purity: 0.38062721490859985
INFO:root:Loss: 0.49475422501564026
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021118']
INFO:root:Efficiency: 0.8317769169807434
INFO:root:Purity: 0.4135875701904297
INFO:root:Loss: 0.4957301914691925
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-c

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.4881, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,   530,  1268, ..., 11245, 11628, 11807]]),
 'truth': array([False, False, False, ..., False, False,  True]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.4962, device='cuda:0'),
 'preds': array([[    0,     0,     1, ..., 10327, 10327, 10328],
       [   12,  1966,     2, ..., 10017, 10046, 10048]]),
 'truth': array([False,  True, False, ..., False,  True,  True]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.4009685051097785 0.8274509852476453


...

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.1124815859183147 0.9900202564372422
Seed solver complete, radius: 1.7386286413561776


INFO:root:Efficiency: 0.9853880405426025
INFO:root:Purity: 0.12356926500797272
INFO:root:Loss: 0.14882905781269073
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021132']
INFO:root:Efficiency: 0.9925702214241028
INFO:root:Purity: 0.12480109930038452
INFO:root:Loss: 0.14931844174861908
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021129']
INFO:root:Efficiency: 0.9896636009216309
INFO:root:Purity: 0.1060699000954628
INFO:root:Loss: 0.13666938245296478
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021056']
INFO:root:Efficiency: 0.9912760257720947
INFO:root:Purity: 0.09762370586395264
INFO:root:Loss: 0.13143736124038696
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021118']
INFO:root:Efficiency: 0.9913901686668396
INFO:root:Purity: 0.11282627284526825
INFO:root:Loss: 0.14211198687553406
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trac

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(0.1488, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 12138, 12138, 12138],
       [    1,   159,   217, ..., 11792, 11807, 11989]]),
 'truth': array([False, False, False, ..., False,  True, False]),
 'truth_graph': array([[  60,   60,  133, ..., 4898, 6374, 7943],
       [ 606,  662,  606, ..., 3861, 4898, 6374]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'loss': tensor(0.1493, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 10328, 10328, 10328],
       [   12,   754,  1359, ..., 10223, 10235, 10261]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[2662, 2662, 4038, ..., 9154, 9520, 9805],
       [4038, 4068, 4771, ..., 8278, 9154, 9520]])}
--------------------------------------------------------------------------------
DATAL

0.11257608413646802 0.99002260237688


In [ ]:
wandb_logger.log_metrics({"99_purity": pur})

In [10]:
pur

0.11557295063784898